In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers, losses , metrics
import seaborn as sns
import matplotlib.pyplot as plt

# System libraries
from pathlib import Path
import os.path

# Metrics
from sklearn.metrics import classification_report, confusion_matrix
import itertools
import random

In [ ]:
seed_value = 0
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
random.seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

In [ ]:
data = "../input/seaanimals"

In [ ]:
image_dir = Path(data)

# Get filepaths and labels
filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.png')) + list(image_dir.glob(r'**/*.PNG'))

labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels], axis=1)
image_df = image_df.groupby('Label').apply(lambda x: x.sample(n=min(498, len(x)))).reset_index(drop=True)

In [ ]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError

path = Path("../input/sea-animals-image-dataste").rglob("*.jpg")
for img_p in path:
    try:
        img = PIL.Image.open(img_p)
    except PIL.UnidentifiedImageError:
            print(img_p)

In [ ]:
image_df

In [ ]:
# Display 16 picture of the dataset with their labels
random_index = np.random.randint(0, len(image_df), 16)
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(10, 10),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(image_df.Filepath[random_index[i]]))
    ax.set_title(image_df.Label[random_index[i]])
plt.tight_layout()
plt.show()

In [ ]:
sns.countplot(data=image_df, x='Label')

In [ ]:
# Set random seed for reproducibility
seed_value = 42

# 1. Set the `PYTHONHASHSEED` environment variable
os.environ['PYTHONHASHSEED'] = str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 3. Set the `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,
                                  shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, fill_mode = 'nearest',
    validation_split=0.2) # set validation split

In [ ]:
train_images = train_datagen.flow_from_directory(
    data,
    target_size=(224, 224),
    batch_size=48,
    class_mode='categorical',
    subset='training') # set as training data

validation_images = train_datagen.flow_from_directory(
    data , # same directory as training data
    target_size=(224, 224),
    batch_size=48,
    class_mode='categorical',
    subset='validation') # set as validation data

In [ ]:
mobile_model = Sequential()

pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)
pretrained_model.trainable = False
mobile_model.add(pretrained_model)
mobile_model.add(Flatten())
mobile_model.add(Dense(512, activation='relu'))
mobile_model.add(Dropout(0.5))
mobile_model.add(Dense(6, activation='softmax'))

In [ ]:
mobile_model.summary()

In [ ]:
mobile_model.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy' , metrics.Precision(), metrics.Recall()])

# history = mobile_model.fit(train_images,
#     steps_per_epoch=len(train_images),
#     validation_data=validation_images,
#     validation_steps=len(validation_images),
#     epochs=10)

In [ ]:
from sklearn.model_selection import StratifiedKFold
all_validation_accuracy = []
all_validation_loss = []
all_precision = []
all_recall = []
# Define the number of folds
num_folds = 3  # You can change this value based on your preference

# Create a StratifiedKFold object
kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=seed_value)

# Iterate over the folds
fold = 0
for train_index, val_index in kf.split(image_df.Filepath, image_df.Label):
    fold += 1
    print(f"Training on Fold {fold}")
    history = mobile_model.fit(train_images, steps_per_epoch=len(train_images), validation_data=validation_images, validation_steps=len(validation_images), epochs=20)

    # Store validation accuracy and loss
    validation_loss, validation_accuracy , precision , recall= mobile_model.evaluate(validation_images, steps=len(validation_images))
    all_validation_accuracy.append(validation_accuracy)
    all_validation_loss.append(validation_loss)
    all_precision.append(precision)
    all_recall.append(precision)

    # Plot the accuracy/loss graphs for each fold
    history_dict = history.history
    loss_values = history_dict['loss']
    val_loss_values = history_dict['val_loss']
    accuracy = history_dict['accuracy']
    val_accuracy = history_dict['val_accuracy']

    epochs = range(1, len(loss_values) + 1)
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))

    # Plot the model accuracy vs Epochs
    ax[0].plot(epochs, accuracy, 'r', label='Training accuracy')
    ax[0].plot(epochs, val_accuracy, 'b', label='Validation accuracy')
    ax[0].set_title(f'Training & Validation Accuracy - Fold {fold}', fontsize=16)
    ax[0].set_xlabel('Epochs', fontsize=16)
    ax[0].set_ylabel('Accuracy', fontsize=16)
    ax[0].legend()

    # Plot the loss vs Epochs
    ax[1].plot(epochs, loss_values, 'r', label='Training loss')
    ax[1].plot(epochs, val_loss_values, 'b', label='Validation loss')
    ax[1].set_title(f'Training & Validation Loss - Fold {fold}', fontsize=16)
    ax[1].set_xlabel('Epochs', fontsize=16)
    ax[1].set_ylabel('Loss', fontsize=16)
    ax[1].legend()

In [ ]:
best_fold = np.argmax(all_validation_accuracy) + 1  # +1 because folds are 1-indexed
print(f'Best Fold: {best_fold}')

In [ ]:
best_fold_history = history

In [ ]:
# Plot the accuracy/loss graphs for the best fold
history_dict = best_fold_history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
accuracy = history_dict['accuracy']
val_accuracy = history_dict['val_accuracy']

epochs = range(1, len(loss_values) + 1)
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Plot the model accuracy vs Epochs
ax[0].plot(epochs, accuracy, 'r', label='Training accuracy')
ax[0].plot(epochs, val_accuracy, 'b', label='Validation accuracy')
ax[0].set_title(f'Training & Validation Accuracy - Best Fold ({best_fold})', fontsize=16)
ax[0].set_xlabel('Epochs', fontsize=16)
ax[0].set_ylabel('Accuracy', fontsize=16)
ax[0].legend()

# Plot the loss vs Epochs
ax[1].plot(epochs, loss_values, 'r', label='Training loss')
ax[1].plot(epochs, val_loss_values, 'b', label='Validation loss')
ax[1].set_title(f'Training & Validation Loss - Best Fold ({best_fold})', fontsize=16)
ax[1].set_xlabel('Epochs', fontsize=16)
ax[1].set_ylabel('Loss', fontsize=16)
ax[1].legend()

plt.show()

In [ ]:
mean_accuracy = np.mean(all_validation_accuracy)
mean_precision = np.mean(all_precision)
mean_recall = np.mean(all_recall)

print(f'Mean Accuracy: {mean_accuracy}')
print(f'Mean Precision: {mean_precision}')
print(f'Mean Recall: {mean_recall}')

In [ ]:
from sklearn.metrics import classification_report
predictions = mobile_model.predict(validation_images, steps=len(validation_images))
predicted_labels = np.argmax(predictions, axis=1)

true_labels = validation_images.classes
# Print classification report
print(classification_report(true_labels, predicted_labels, target_names=train_images.class_indices.keys()))


In [ ]:
# from sklearn.metrics import confusion_matrix
# import seaborn as sns

# predictions = mobile_model.predict(validation_images, steps=len(validation_images))
# predicted_labels = np.argmax(predictions, axis=1)

# true_labels = validation_images.classes

# conf_matrix = confusion_matrix(true_labels, predicted_labels)

# plt.figure(figsize=(8, 8))
# sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
#             xticklabels=validation_images.class_indices.keys(),
#             yticklabels=validation_images.class_indices.keys())
# plt.title('Confusion Matrix')
# plt.xlabel('Predicted Labels')
# plt.ylabel('True Labels')
# plt.show()
